### Importing required libraries

In [1]:
import cv2
import copy
import math
import numpy as np

### Finding tips of fingers and Middle points between fingers 

In [2]:
def tips_mid_coordsFingers(res, drawing):
    #  convexity defect
    defects_points = []
    start_points = []
    hull = cv2.convexHull(res, returnPoints=False)
    if len(hull) > 3:
        defects = cv2.convexityDefects(res, hull)
        if defects is not None:
            cnt = 0
            for i in range(defects.shape[0]):  # calculate the angle
                #calculation of components of convex defects 
                s, e, f, d = defects[i][0]
                start = tuple(res[s][0])
                end = tuple(res[e][0])
                far = tuple(res[f][0])
                start_points.append(start)
                cv2.circle(drawing, far, 4, [211, 84, 0], -1)
                cv2.circle(drawing, start, 4, [155, 0, 255], -1)
                a = math.sqrt((end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2)
                b = math.sqrt((far[0] - start[0]) ** 2 + (far[1] - start[1]) ** 2)
                c = math.sqrt((end[0] - far[0]) ** 2 + (end[1] - far[1]) ** 2)
                angle = math.acos((b ** 2 + c ** 2 - a ** 2) / (2 * b * c))  # cosine theorem
                if angle <= math.pi / 2:  # angle less than 90 degree, treat as fingers
                    defects_points.append(far)
    return start_points, defects_points

### Finding approx. size of finger, where ring can be worn

In [3]:
def sizeFinger(points_btwn_fingers):
    #considering first 3 extreme points
    size_ringFinger = points_btwn_fingers[0][0] - points_btwn_fingers[1][0]
    size_middleFinger = points_btwn_fingers[1][0] - points_btwn_fingers[2][0]
    #approximate finger size
    sum_FingerSize = size_ringFinger+size_middleFinger
    approx_FingerSize = sum_FingerSize//2
    return approx_FingerSize

### Finding approx. depth on finger, where ring is worn

In [4]:
def depthFinger(points_btwn_fingers):
    #diff Y-coords of two corners of finger
    diff_ring_finger_Y = points_btwn_fingers[0][1] - points_btwn_fingers[1][1]
    '''as per Y-coords of two corners of middle finger, has no much difference,
        observing the graph and fact of positioning of fingers, taking left Y-coordinate of index finger same 
        as right Y-coordinate of index finger
    '''
    depthFinger_list = [points_btwn_fingers[0][1]+diff_ring_finger_Y,points_btwn_fingers[2][1],points_btwn_fingers[3][1]+diff_ring_finger_Y]
    return depthFinger_list

### Finding approx. two end coordinates of each finger

In [5]:
def getTwoCornerPoints_Finger(points_btwn_fingers,approx_FingerSize,littleFinger_Y,indexFinger_Y,thumbFinger_Y):
    #little finger
    little_finger_X = points_btwn_fingers[0][0]+approx_FingerSize
    little_finger_Y = littleFinger_Y
    little_finger_corner_coords = [little_finger_X,little_finger_Y]
    #index finger
    index_finger_X = points_btwn_fingers[2][0]-approx_FingerSize
    index_finger_Y = indexFinger_Y
    index_finger_corner_coords = [index_finger_X,index_finger_Y]
    #thumb finger
    thumb_finger_X = points_btwn_fingers[3][0]-approx_FingerSize
    thumb_finger_Y = thumbFinger_Y
    thumb_finger_corner_coords = [thumb_finger_X,thumb_finger_Y]
    #all coords in order
    all_coords_fingers_in_order = [little_finger_corner_coords,points_btwn_fingers[0],points_btwn_fingers[1],points_btwn_fingers[2],index_finger_corner_coords,points_btwn_fingers[3],thumb_finger_corner_coords]
    return all_coords_fingers_in_order

### Finding approx. mid-points of each finger

In [6]:
def getMidPointsFingers(getTwoCornerPoints_Finger_list):
    #Little Finger
    midPoint_LittleFinger_X = (getTwoCornerPoints_Finger_list[0][0]+getTwoCornerPoints_Finger_list[1][0]) // 2
    midPoint_LittleFinger_Y = (getTwoCornerPoints_Finger_list[0][1]+getTwoCornerPoints_Finger_list[1][1]) // 2
    midPoint_LittleFinger_coords = [midPoint_LittleFinger_X,midPoint_LittleFinger_Y]
    #Ring Finger
    midPoint_RingFinger_X = (getTwoCornerPoints_Finger_list[1][0]+getTwoCornerPoints_Finger_list[2][0]) // 2
    midPoint_RingFinger_Y = (getTwoCornerPoints_Finger_list[1][1]+getTwoCornerPoints_Finger_list[2][1]) // 2
    midPoint_RingFinger_coords = [midPoint_RingFinger_X,midPoint_RingFinger_Y]
    #Middle Finger
    midPoint_MiddleFinger_X = (getTwoCornerPoints_Finger_list[2][0]+getTwoCornerPoints_Finger_list[3][0]) // 2
    midPoint_MiddleFinger_Y = (getTwoCornerPoints_Finger_list[2][1]+getTwoCornerPoints_Finger_list[3][1]) // 2
    midPoint_MiddleFinger_coords = [midPoint_MiddleFinger_X,midPoint_MiddleFinger_Y]
    #Index Finger
    midPoint_IndexFinger_X = (getTwoCornerPoints_Finger_list[3][0]+getTwoCornerPoints_Finger_list[4][0]) // 2
    midPoint_IndexFinger_Y = (getTwoCornerPoints_Finger_list[3][1]+getTwoCornerPoints_Finger_list[4][1]) // 2
    midPoint_IndexFinger_coords = [midPoint_IndexFinger_X,midPoint_IndexFinger_Y]
    #Thumb Finger
    midPoint_ThumbFinger_X = (getTwoCornerPoints_Finger_list[5][0]+getTwoCornerPoints_Finger_list[6][0]) // 2
    midPoint_ThumbFinger_Y = (getTwoCornerPoints_Finger_list[5][1]+getTwoCornerPoints_Finger_list[6][1]) // 2
    midPoint_ThumbFinger_coords = [midPoint_ThumbFinger_X,midPoint_ThumbFinger_Y]
    midPoints_all_fingers_list = [midPoint_LittleFinger_coords,midPoint_RingFinger_coords,midPoint_MiddleFinger_coords,midPoint_IndexFinger_coords,midPoint_ThumbFinger_coords]
    return midPoints_all_fingers_list

### Read and Display an image

In [7]:
img = cv2.imread('my_hand.jpg')
cv2.imshow('My Original Image',img)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Background removal
removes background (if,any)

In [8]:
kernel = np.ones((3,3), np.uint8)
img = cv2.erode(img, kernel, iterations=1)

### Skin detection and Thresholding for masked View

In [9]:
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
lower = np.array([0, 48, 80], dtype="uint8")
upper = np.array([20, 255, 255], dtype="uint8")
skinMask = cv2.inRange(hsv, lower, upper)
cv2.imshow('Threshold Hands', skinMask)
cv2.waitKey(0)
cv2.destroyAllWindows()

### Getting the contours and convex hull of a hand

In [10]:
skinMask_copy = copy.deepcopy(skinMask)
contours, hierarchy = cv2.findContours(skinMask_copy, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
length = len(contours)
"""Find the largest contour"""
maxArea = -1
if length > 0:
    for i in range(length):
        temp = contours[i]
        area = cv2.contourArea(temp)
        if area > maxArea:
            maxArea = area
            ci = i

    res = contours[ci]
    #computing hull points
    hull = cv2.convexHull(res)
    drawing = np.zeros(img.shape, np.uint8)
    cv2.drawContours(drawing, [res], 0, (0, 255, 0), 2)#contours of hand (green)
    cv2.drawContours(drawing, [hull], 0, (0, 0, 255), 3)#shape of convex hull around hand(red)
    
    start_points, defects_points = tips_mid_coordsFingers(res, drawing)
    print("fingertips points", start_points)
    print("points/defects in between fingers", defects_points)
    cv2.imshow('convex hull, defect points and start points', drawing)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

fingertips points [(100, 35), (84, 16), (64, 4), (36, 5), (32, 6), (5, 52), (4, 55), (76, 179)]
points/defects in between fingers [(76, 56), (63, 49), (52, 47), (35, 67), (6, 53)]


In [11]:
'''filtering start_points and defect_points based on physiological structure,
    these points has to be changed based on the fingertips point vallues and points/defects
    in between fingers values
'''
tips_fingers = [(100, 35), (84, 16), (64, 4), (36,5), (5, 52)]
#points_btwn_fingers
points_btwn_fingers = [(76, 56), (63, 49), (52, 47), (35, 67)]
print("points_btwn_fingers", points_btwn_fingers)

#getting the approximate size of each finger considersing middle and ring finger
approx_FingerSize = sizeFinger(points_btwn_fingers)

#depth/ Y_coords of fingers
littleFinger_Y,indexFinger_Y,thumbFinger_Y = depthFinger(points_btwn_fingers) 

#approximate coordinate values of two corners of each finger where ring can be worn
getTwoCornerPoints_Finger_list = getTwoCornerPoints_Finger(points_btwn_fingers,approx_FingerSize,littleFinger_Y,indexFinger_Y,thumbFinger_Y)
print("getTwoCornerPoints_Finger_list",getTwoCornerPoints_Finger_list)

#mid-points of each finger
getMidPointsFingers_list = getMidPointsFingers(getTwoCornerPoints_Finger_list) 
print("getMidPointsFingers_list",getMidPointsFingers_list)

points_btwn_fingers [(76, 56), (63, 49), (52, 47), (35, 67)]
getTwoCornerPoints_Finger_list [[88, 63], (76, 56), (63, 49), (52, 47), [40, 47], (35, 67), [23, 74]]
getMidPointsFingers_list [[82, 59], [69, 52], [57, 48], [46, 47], [29, 70]]


### Coordinates list

In [12]:
#coords of little finger
print("Coordinates of little finger are as follows:")
print("Right:",getTwoCornerPoints_Finger_list[0])
print("Middle:",getMidPointsFingers_list[0])
print("Left:",getTwoCornerPoints_Finger_list[1])

#coords of ring finger
print("Coordinates of ring finger are as follows:")
print("Right:",getTwoCornerPoints_Finger_list[1])
print("Middle:",getMidPointsFingers_list[1])
print("Left:",getTwoCornerPoints_Finger_list[2])

#coords of middle finger
print("Coordinates of middle finger are as follows:")
print("Right:",getTwoCornerPoints_Finger_list[2])
print("Middle:",getMidPointsFingers_list[2])
print("Left:",getTwoCornerPoints_Finger_list[3])

#coords of index finger
print("Coordinates of index finger are as follows:")
print("Right:",getTwoCornerPoints_Finger_list[3])
print("Middle:",getMidPointsFingers_list[3])
print("Left:",getTwoCornerPoints_Finger_list[4])

 #coords of thumb finger
print("Coordinates of thumb finger are as follows:")
print("Right:",getTwoCornerPoints_Finger_list[5])
print("Middle:",getMidPointsFingers_list[4])
print("Left:",getTwoCornerPoints_Finger_list[6])

Coordinates of little finger are as follows:
Right: [88, 63]
Middle: [82, 59]
Left: (76, 56)
Coordinates of ring finger are as follows:
Right: (76, 56)
Middle: [69, 52]
Left: (63, 49)
Coordinates of middle finger are as follows:
Right: (63, 49)
Middle: [57, 48]
Left: (52, 47)
Coordinates of index finger are as follows:
Right: (52, 47)
Middle: [46, 47]
Left: [40, 47]
Coordinates of thumb finger are as follows:
Right: (35, 67)
Middle: [29, 70]
Left: [23, 74]


### Verifiying ranges of resulted coordinates
Displaying the pixels of image with the minimum and maximum points from coordinates lists above 

In [13]:
img = cv2.imread('my_hand.jpg')
#ROI by defining pixel value
# (20,90 )(35, 88)
fingers_output = img[47:85, 20:85]
cv2.imshow('output', fingers_output)
cv2.waitKey(0)
cv2.destroyAllWindows()